# Covid-19 data download & processing
---
This Python Script downloads an up-to-date dataset for **Covid-19** and exports the data to the data folder.

The Data is from the **R**obert **K**och **I**nstitut downloaded over [ArcGis Hub](https://hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6?page=15976).

*Script was created on Python: 3.7.6 64-bit Kernel*

In [1]:
import pandas as pd
import math

import io               # file operations
import json

import ssl              # secure client-server connection
import requests         # html-requests

In [2]:
# Uncomment next 2 lines to install jsonmerge
#import sys
#!{sys.executable} -m pip install jsonmerge
from jsonmerge import Merger

In [3]:
sourceURL = 'https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query?'
objectIdsQuery = 'where=1%3D1&returnIdsOnly=true&f=json'
dataSetQuery = 'where=ObjectId+BETWEEN+0+AND+0' # just an example gets created later dynamically
dataQuery = '&outSR=4326&outFields=IdBundesland,Bundesland,Landkreis,Altersgruppe,AnzahlFall,AnzahlTodesfall,ObjectId,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn&f=json'

## Requesting which Features (ObjectID´s) are available

In [4]:
objectIdsRequest = requests.get(sourceURL + objectIdsQuery)
objectIdsRequest.status_code

200

In [5]:
objectIds = json.loads(objectIdsRequest.text)

numOfObjectIds = len(objectIds['objectIds'])

objectIdStart = objectIds['objectIds'][0]
objectIdEnd = objectIds['objectIds'][numOfObjectIds - 1]
print(f'Range of ObjectIds: [{objectIdStart}, {objectIdEnd}]')

Range of ObjectIds: [33350699, 33551886]


## Requesting Features

In [6]:
dataRequest = requests.get(sourceURL + 'where=1%3D1' + dataQuery)
dataRequest.status_code

200

In [7]:
data = json.loads(dataRequest.text)
maxApiRequest = len(data['features'])

neededRequests = math.ceil(numOfObjectIds / maxApiRequest)

print(f'The download will require {neededRequests - 1} more requests due to the server limit of {maxApiRequest} features/request.')

The download will require 40 more requests due to the server limit of 5000 features/request.


In [8]:
# Json-Merger with custom rule
jsonMergeSchema = {"properties":{"features":{"mergeStrategy":"append"}}}
dataMerger = Merger(jsonMergeSchema)

In [9]:
i = 0
rangeLowerEnd = data['features'][maxApiRequest - 1]['attributes']['ObjectId'] + 1
rangeUpperEnd = rangeLowerEnd + maxApiRequest

while (i < neededRequests - 1): # neededRequests - 1 because of initial download
    dataSetQuery = f'where=ObjectId+BETWEEN+{rangeLowerEnd}+AND+{rangeUpperEnd}'
    temp_sourceURL = sourceURL + dataSetQuery + dataQuery
    print(i, f'Pulling ObjectIds: [{rangeLowerEnd}, {rangeUpperEnd}]')

    temp_dataRequest = requests.get(temp_sourceURL)
    if (temp_dataRequest.status_code > 200): # stop when a request isn´t working
        print(f'Error in request: {temp_dataRequest.status_code}')
        break
    temp_data = json.loads(temp_dataRequest.text)

    # append new data to already downloaded one
    data = dataMerger.merge(data, temp_data)

    temp_dataLength = len(data['features'])
    t_le = data['features'][0]['attributes']['ObjectId']
    t_ue = data['features'][temp_dataLength - 1]['attributes']['ObjectId']
    print(f'Total collected features: {temp_dataLength} From ObjectIds: [{t_le}, {t_ue}]')

    rangeLowerEnd = rangeUpperEnd + 1
    rangeUpperEnd += maxApiRequest + 1
    if (rangeUpperEnd > objectIdEnd):
        rangeUpperEnd = objectIdEnd
    i += 1

print('Done')

0 Pulling ObjectIds: [33355708, 33360708]
Total collected features: 10000 From ObjectIds: [33350699, 33360707]
1 Pulling ObjectIds: [33360709, 33365709]
Total collected features: 15000 From ObjectIds: [33350699, 33365708]
2 Pulling ObjectIds: [33365710, 33370710]
Total collected features: 20000 From ObjectIds: [33350699, 33370709]
3 Pulling ObjectIds: [33370711, 33375711]
Total collected features: 25000 From ObjectIds: [33350699, 33375710]
4 Pulling ObjectIds: [33375712, 33380712]
Total collected features: 30000 From ObjectIds: [33350699, 33380711]
5 Pulling ObjectIds: [33380713, 33385713]
Total collected features: 35000 From ObjectIds: [33350699, 33385712]
6 Pulling ObjectIds: [33385714, 33390714]
Total collected features: 40000 From ObjectIds: [33350699, 33390713]
7 Pulling ObjectIds: [33390715, 33395715]
Total collected features: 45000 From ObjectIds: [33350699, 33395714]
8 Pulling ObjectIds: [33395716, 33400716]
Total collected features: 50000 From ObjectIds: [33350699, 33400715]
9

In [10]:
print('Entries: ', len(data['features']))
print('Structure: ', data['features'][0])
print('Latest data: ', data['features'][0]['attributes']['Datenstand'])

Entries:  201140
Structure:  {'attributes': {'IdBundesland': 1, 'Bundesland': 'Schleswig-Holstein', 'Landkreis': 'SK Flensburg', 'Altersgruppe': 'A00-A04', 'AnzahlFall': 1, 'AnzahlTodesfall': 0, 'ObjectId': 33350699, 'Meldedatum': 1598227200000, 'IdLandkreis': '01001', 'Datenstand': '06.09.2020, 00:00 Uhr', 'NeuerFall': 0, 'NeuerTodesfall': -9, 'Refdatum': 1598227200000, 'NeuGenesen': -9, 'AnzahlGenesen': 0, 'IstErkrankungsbeginn': 0}}
Latest data:  06.09.2020, 00:00 Uhr


In [11]:
def display_n(df,n): 
    with pd.option_context('display.max_rows',n*2):
        display(df)

In [12]:
dfx = pd.DataFrame.from_dict(data['features'])
display_n(dfx, 2)

,attributes
0,"{'IdBundesland': 1, 'Bundesland': 'Schleswig-H..."
1,"{'IdBundesland': 1, 'Bundesland': 'Schleswig-H..."
...,...
201138,"{'IdBundesland': 16, 'Bundesland': 'Thüringen'..."
201139,"{'IdBundesland': 16, 'Bundesland': 'Thüringen'..."


In [13]:
# turning the collumn attributes in seperated collumns
for rowid in data['fields']:
    dfx[rowid['name']] = dfx.apply(lambda row: row.loc['attributes'][rowid['name']], axis=1)
dfx = dfx.drop(['attributes'], axis=1)
display_n(dfx, 2)

,IdBundesland,Bundesland,Landkreis,Altersgruppe,AnzahlFall,AnzahlTodesfall,ObjectId,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn
0,1,Schleswig-Holstein,SK Flensburg,A00-A04,1,0,33350699,1598227200000,01001,"06.09.2020, 00:00 Uhr",0,-9,1598227200000,-9,0,0
1,1,Schleswig-Holstein,SK Flensburg,A05-A14,1,0,33350700,1597449600000,01001,"06.09.2020, 00:00 Uhr",0,-9,1597449600000,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201138,16,Thüringen,LK Altenburger Land,A80+,2,0,33551885,1590624000000,16077,"06.09.2020, 00:00 Uhr",0,-9,1590624000000,0,2,0
201139,16,Thüringen,LK Altenburger Land,A80+,1,0,33551886,1591660800000,16077,"06.09.2020, 00:00 Uhr",0,-9,1591660800000,0,1,0


In [14]:
data['features'][0]['attributes']['Landkreis']

'SK Flensburg'

In [15]:
frameByLK = dfx.groupby(['Landkreis', 'IdLandkreis'])['AnzahlFall', 'AnzahlTodesfall', 'AnzahlGenesen'].sum().reset_index().set_index('Landkreis')

In [16]:
frameByLK

,IdLandkreis,AnzahlFall,AnzahlTodesfall,AnzahlGenesen
Landkreis,,,,
LK Ahrweiler,07131,319,2,268
LK Aichach-Friedberg,09771,451,20,407
LK Alb-Donau-Kreis,08425,792,26,704
LK Altenburger Land,16077,98,4,75
LK Altenkirchen,07132,210,10,187
...,...,...,...,...
SK Worms,07319,273,8,250
SK Wuppertal,05124,1461,86,1256
SK Würzburg,09663,544,52,450


In [17]:
frameByLK.to_csv('frameByLK.csv', index=True, encoding='utf-8')

In [18]:
dfx_slim = dfx.drop(columns=['IdBundesland', 'Bundesland', 'ObjectId', 'NeuerFall', 'NeuerTodesfall', 'IstErkrankungsbeginn', 'NeuGenesen', 'Meldedatum', 'Datenstand', 'Landkreis']).groupby(['Refdatum', 'IdLandkreis', 'Altersgruppe']).sum().groupby(['Refdatum', 'IdLandkreis']).sum()
dfx_slim = dfx_slim.groupby(['IdLandkreis', 'Refdatum']).sum().groupby(level=[0]).cumsum()
dfx_slim['acute'] = dfx_slim['AnzahlFall'] - (dfx_slim['AnzahlTodesfall'] + dfx_slim['AnzahlGenesen'])
display_n(dfx_slim, 3)

AnzahlFall  AnzahlTodesfall  AnzahlGenesen  acute
IdLandkreis Refdatum                                                        
01001       1583798400000           1                0              1      0
            1583884800000           2                0              2      0
            1583971200000           3                0              3      0
...                               ...              ...            ...    ...
16077       1598659200000          94                4             75     15
            1598918400000          97                4             75     18
            1599091200000          98                4             75     19

[44406 rows x 4 columns]

In [19]:
dfx_slim.to_csv('timeFrameByLK.csv', index=True, encoding='utf-8')